> ## Import Package

In [27]:
import os
import torch
import numpy as np
import pandas as pd
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

> ## Define Constants
> If DEBUG_TRAIN is true, shows the shape of the tensor during the learning process.<br>
> If DEBUG_DATA is true, shows the data during the learning process.<br>
> MODEL_PATH specifies the path, which models completed learning will be stored.

In [40]:
DEBUG_TRAIN = False
DEBUG_DATA = False

MODEL_PATH = './saved_model/BCEWithLogitsLoss-Complete.torchModel'
CSV_FILE = "./image-data/labels-map.csv"
ROOT_DIR = "./image-data/hangul-images"

batch_size=512
learning_rate=0.002
num_epoch=100

> ## Define Functions
> **encode_from_utf8**: returns a list of bit that converted from character represented by UTF-8<br>
> **decode_from_bin**: returns a character represented by UTF-8 that converted from list of bit<br>

In [41]:
def encode_from_utf8(x):
    return list(map(float,bin(int(x.encode().hex(),16))[2:]))

In [42]:
def decode_from_bin(x):
    res = ""
    for i in x:
        res+=str(round(i))
        pass
    return bytearray.fromhex(hex(int(res, 2))[2:]).decode()

In [43]:
def decode_from_bin2(x):
    res = ""
    for i in x:
        res+=str(int(i>0))
        pass
    return bytearray.fromhex(hex(int(res, 2))[2:]).decode()

> ## Define Dataset Class

In [44]:
class KoreanHandwritingDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.dataset = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.dataset.iloc[idx, 0]
        image = io.imread(img_name)
        label = self.dataset.iloc[idx, 1]
        label = np.array([label])
        sample = {'image': image, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample
    pass

> ## Define Transform Class

In [45]:
class ToTensor(object):
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        image = image/255.0
        return {'image': torch.from_numpy(image.reshape(1,64,64)),
                'label': torch.tensor(encode_from_utf8(label[0]))}

In [46]:
korean_dataset = KoreanHandwritingDataset(CSV_FILE, ROOT_DIR, transform=transforms.Compose([ToTensor()]))
dataloader = DataLoader(korean_dataset, batch_size = batch_size, shuffle = True, num_workers = 0)

> ## Define Model

In [47]:
class CNN(nn.Module):
  
    def __init__(self):
        super(CNN,self).__init__()
        self.avgpool1 = nn.AvgPool2d(2,2) # kernel size 2x2 (32 = 64/2)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(2,2) # kernel size 2x2 (32 = 64/2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(2,2) # kernel size 2x2 (16 = 32/2)
        self.linear1 = nn.Linear(1152,378)
        self.relu3 = nn.ReLU()
        self.linear2 = nn.Linear(378,128)
        self.relu4 = nn.ReLU()
        self.linear3 = nn.Linear(128,24)
        #self.sigmoid = nn.Sigmoid()
        pass
    def forward(self,x):
        out = self.avgpool1(x)
        if DEBUG_TRAIN : print('avgpool1: ', out.shape)
        out = self.conv1(out)
        if DEBUG_TRAIN : print('conv1: ', out.shape)
        out = self.relu1(out)
        if DEBUG_TRAIN : print('relu1: ', out.shape)
        out = self.maxpool1(out)
        if DEBUG_TRAIN : print('maxpool1: ', out.shape)
        out = self.conv2(out)
        if DEBUG_TRAIN : print('conv2: ', out.shape)
        out = self.relu2(out)
        if DEBUG_TRAIN : print('relu2: ', out.shape)
        out = self.maxpool2(out)
        if DEBUG_TRAIN : print('maxpool2: ', out.shape)
        out = out.view(out.size(0),-1)
        if DEBUG_TRAIN : print('view: ', out.shape)
        out = self.linear1(out)
        if DEBUG_TRAIN : print('linear1: ', out.shape)
        out = self.relu3(out)
        if DEBUG_TRAIN : print('relu3: ', out.shape)
        out = self.linear2(out)
        if DEBUG_TRAIN : print('linear2: ', out.shape)
        out = self.relu4(out)
        if DEBUG_TRAIN : print('relu4: ', out.shape)
        out = self.linear3(out)
        if DEBUG_TRAIN : print('linear3: ', out.shape)
        #out = self.sigmoid(out)
        #if DEBUG_TRAIN : print('sigmoid: ', out.shape)
        if DEBUG_TRAIN : print()
        return out

> ## Create Model
> if saved model exist, load model from disk

In [48]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model=CNN().double().to(device)
if(os.path.isfile(MODEL_PATH)):
    model.load_state_dict(torch.load(MODEL_PATH))
    print(MODEL_PATH, ' loaded!')
    model.eval()
    pass    
loss_function=nn.BCEWithLogitsLoss()
optimizer= torch.optim.Adam(model.parameters(),lr=learning_rate)

cuda:0
./saved_model/BCEWithLogitsLoss-Complete.torchModel  loaded!


> ## Train Model

In [49]:
loss_arr=[]
for i in range(num_epoch):
    for j, sample in enumerate(dataloader):
        x = sample['image'].to(device)
        y = sample['label'].to(device)
        optimizer.zero_grad()
        output=model.forward(x)
        loss=loss_function(output,y)
        loss.backward()
        optimizer.step()
        if j == 0:
            if DEBUG_DATA:
                plt.figure(figsize=(32, 32))
                for k, img in enumerate(x):
                    plt.subplot(8, 8, k+1)
                    plt.imshow(img.squeeze())
                    plt.xticks([])
                    plt.yticks([])
                    pass
                plt.show()
            print('epoch: ', i+1, 'sequence: ', j, loss)
            loss_arr.append(loss.detach().cpu().numpy())

epoch:  1 sequence:  0 tensor(0.2106, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  2 sequence:  0 tensor(0.1112, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  3 sequence:  0 tensor(0.0753, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  4 sequence:  0 tensor(0.0634, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  5 sequence:  0 tensor(0.0495, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  6 sequence:  0 tensor(0.0479, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  7 sequence:  0 tensor(0.0443, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  8 sequence:  0 tensor(0.0384, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  9 sequence:  0 tensor(0.0357, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  10 sequence:  0 tensor(0.0365, device='cuda:0', grad_fn=<BinaryCr

epoch:  79 sequence:  0 tensor(3.6634e-05, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  80 sequence:  0 tensor(3.3504e-05, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  81 sequence:  0 tensor(3.0021e-05, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  82 sequence:  0 tensor(2.5538e-05, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  83 sequence:  0 tensor(1.9300e-05, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  84 sequence:  0 tensor(2.5964e-05, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  85 sequence:  0 tensor(2.0612e-05, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  86 sequence:  0 tensor(2.1353e-05, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  87 sequence:  0 tensor(2.0018e-05, device='cuda:0',
       grad_fn=<Bina

> ## Save Current Model

In [13]:
torch.save(model.state_dict(), MODEL_PATH)

> ## Test Model

In [51]:
with torch.no_grad():
    scoreboard = []
    for j, sample in enumerate(dataloader):
        x = sample['image'].to(device)
        y = sample['label'].to(device)
        output = model.forward(x)
        for i in range(len(output)):
            try:
                out = decode_from_bin2(output[i].tolist())
            except UnicodeDecodeError:
                out = 'UnicodeDecodeError'
            y_ = decode_from_bin(y[i].tolist())
            print(", 실제 값: ", y_, "출력 값: ", out)
            if out==y_:
                scoreboard.append(1)
            else:
                scoreboard.append(0)

, 실제 값:  삠 출력 값:  삠
, 실제 값:  잊 출력 값:  잊
, 실제 값:  생 출력 값:  생
, 실제 값:  왑 출력 값:  왑
, 실제 값:  궈 출력 값:  궈
, 실제 값:  무 출력 값:  무
, 실제 값:  냄 출력 값:  냄
, 실제 값:  건 출력 값:  건
, 실제 값:  뇨 출력 값:  뇨
, 실제 값:  횔 출력 값:  횔
, 실제 값:  셸 출력 값:  셸
, 실제 값:  흰 출력 값:  흰
, 실제 값:  힝 출력 값:  힝
, 실제 값:  쎄 출력 값:  쎄
, 실제 값:  훌 출력 값:  훌
, 실제 값:  람 출력 값:  람
, 실제 값:  런 출력 값:  런
, 실제 값:  쎄 출력 값:  쎄
, 실제 값:  뵐 출력 값:  뵐
, 실제 값:  텍 출력 값:  텍
, 실제 값:  녁 출력 값:  녁
, 실제 값:  열 출력 값:  열
, 실제 값:  뮷 출력 값:  뮷
, 실제 값:  헙 출력 값:  헙
, 실제 값:  툼 출력 값:  툼
, 실제 값:  밑 출력 값:  밑
, 실제 값:  땃 출력 값:  땃
, 실제 값:  틜 출력 값:  틜
, 실제 값:  뵨 출력 값:  뵨
, 실제 값:  깠 출력 값:  깠
, 실제 값:  벅 출력 값:  벅
, 실제 값:  겄 출력 값:  겄
, 실제 값:  쁘 출력 값:  쁘
, 실제 값:  걋 출력 값:  걋
, 실제 값:  큰 출력 값:  큰
, 실제 값:  증 출력 값:  증
, 실제 값:  석 출력 값:  석
, 실제 값:  썼 출력 값:  썼
, 실제 값:  혭 출력 값:  혭
, 실제 값:  캇 출력 값:  캇
, 실제 값:  해 출력 값:  해
, 실제 값:  쥼 출력 값:  쥼
, 실제 값:  냇 출력 값:  냇
, 실제 값:  택 출력 값:  택
, 실제 값:  관 출력 값:  관
, 실제 값:  쫌 출력 값:  쫌
, 실제 값:  모 출력 값:  모
, 실제 값:  욱 출력 값:  욱
, 실제 값:  뉘 출력 값:  뉘
, 실제 값:  댑 출력 값:  댑


, 실제 값:  뢍 출력 값:  뢍
, 실제 값:  셸 출력 값:  셸
, 실제 값:  름 출력 값:  름
, 실제 값:  랏 출력 값:  랏
, 실제 값:  별 출력 값:  별
, 실제 값:  뵌 출력 값:  뵌
, 실제 값:  통 출력 값:  통
, 실제 값:  듄 출력 값:  듄
, 실제 값:  고 출력 값:  고
, 실제 값:  끔 출력 값:  끔
, 실제 값:  늴 출력 값:  늴
, 실제 값:  첬 출력 값:  첬
, 실제 값:  개 출력 값:  개
, 실제 값:  젤 출력 값:  젤
, 실제 값:  짊 출력 값:  짊
, 실제 값:  땔 출력 값:  땔
, 실제 값:  붸 출력 값:  붸
, 실제 값:  췌 출력 값:  췌
, 실제 값:  육 출력 값:  육
, 실제 값:  괬 출력 값:  괬
, 실제 값:  쐤 출력 값:  쐤
, 실제 값:  꿨 출력 값:  꿨
, 실제 값:  플 출력 값:  플
, 실제 값:  덮 출력 값:  덮
, 실제 값:  옭 출력 값:  옭
, 실제 값:  큭 출력 값:  큭
, 실제 값:  쥴 출력 값:  쥴
, 실제 값:  뻔 출력 값:  뻔
, 실제 값:  믿 출력 값:  믿
, 실제 값:  틱 출력 값:  틱
, 실제 값:  떳 출력 값:  떳
, 실제 값:  뉘 출력 값:  뉘
, 실제 값:  촛 출력 값:  촛
, 실제 값:  삥 출력 값:  삥
, 실제 값:  죙 출력 값:  죙
, 실제 값:  벚 출력 값:  벚
, 실제 값:  쐬 출력 값:  쐬
, 실제 값:  슨 출력 값:  슨
, 실제 값:  첸 출력 값:  첸
, 실제 값:  숨 출력 값:  숨
, 실제 값:  붸 출력 값:  붸
, 실제 값:  캉 출력 값:  캉
, 실제 값:  빻 출력 값:  빻
, 실제 값:  탕 출력 값:  탕
, 실제 값:  룝 출력 값:  룝
, 실제 값:  쇔 출력 값:  쇔
, 실제 값:  퓐 출력 값:  퓐
, 실제 값:  쟬 출력 값:  쟬
, 실제 값:  텬 출력 값:  텬
, 실제 값:  팖 출력 값:  팖


, 실제 값:  않 출력 값:  않
, 실제 값:  햅 출력 값:  햅
, 실제 값:  런 출력 값:  런
, 실제 값:  윕 출력 값:  윕
, 실제 값:  슥 출력 값:  슥
, 실제 값:  흔 출력 값:  흔
, 실제 값:  쏜 출력 값:  쏜
, 실제 값:  줘 출력 값:  줘
, 실제 값:  텁 출력 값:  텁
, 실제 값:  엣 출력 값:  엣
, 실제 값:  튠 출력 값:  튠
, 실제 값:  늦 출력 값:  늦
, 실제 값:  훼 출력 값:  훼
, 실제 값:  브 출력 값:  브
, 실제 값:  핫 출력 값:  핫
, 실제 값:  듐 출력 값:  듐
, 실제 값:  묑 출력 값:  묑
, 실제 값:  꿩 출력 값:  꿩
, 실제 값:  셸 출력 값:  셸
, 실제 값:  회 출력 값:  회
, 실제 값:  둬 출력 값:  둬
, 실제 값:  왼 출력 값:  왼
, 실제 값:  헌 출력 값:  헌
, 실제 값:  얌 출력 값:  얌
, 실제 값:  톨 출력 값:  톨
, 실제 값:  몲 출력 값:  몲
, 실제 값:  랙 출력 값:  랙
, 실제 값:  혭 출력 값:  혭
, 실제 값:  촁 출력 값:  촁
, 실제 값:  넒 출력 값:  넒
, 실제 값:  뀝 출력 값:  뀝
, 실제 값:  읾 출력 값:  읾
, 실제 값:  뉼 출력 값:  뉼
, 실제 값:  샬 출력 값:  샬
, 실제 값:  갱 출력 값:  갱
, 실제 값:  멥 출력 값:  멥
, 실제 값:  늘 출력 값:  늘
, 실제 값:  멩 출력 값:  멩
, 실제 값:  끎 출력 값:  끎
, 실제 값:  셜 출력 값:  셜
, 실제 값:  눌 출력 값:  눌
, 실제 값:  여 출력 값:  여
, 실제 값:  창 출력 값:  창
, 실제 값:  쌔 출력 값:  쌔
, 실제 값:  욘 출력 값:  욘
, 실제 값:  뫄 출력 값:  뫄
, 실제 값:  쫘 출력 값:  쫘
, 실제 값:  탐 출력 값:  탐
, 실제 값:  덕 출력 값:  덕
, 실제 값:  용 출력 값:  용


, 실제 값:  틀 출력 값:  틀
, 실제 값:  퇸 출력 값:  퇸
, 실제 값:  싼 출력 값:  싼
, 실제 값:  곗 출력 값:  곗
, 실제 값:  꽂 출력 값:  꽂
, 실제 값:  엘 출력 값:  엘
, 실제 값:  쐰 출력 값:  쐰
, 실제 값:  킨 출력 값:  킨
, 실제 값:  뭏 출력 값:  뭏
, 실제 값:  틴 출력 값:  틴
, 실제 값:  돌 출력 값:  돌
, 실제 값:  엿 출력 값:  엿
, 실제 값:  멎 출력 값:  멎
, 실제 값:  탭 출력 값:  탭
, 실제 값:  약 출력 값:  약
, 실제 값:  좼 출력 값:  좼
, 실제 값:  성 출력 값:  성
, 실제 값:  꼇 출력 값:  꼇
, 실제 값:  잖 출력 값:  잖
, 실제 값:  때 출력 값:  때
, 실제 값:  썼 출력 값:  썼
, 실제 값:  뮷 출력 값:  뮷
, 실제 값:  홱 출력 값:  홱
, 실제 값:  뺐 출력 값:  뺐
, 실제 값:  뎁 출력 값:  뎁
, 실제 값:  쨩 출력 값:  쨩
, 실제 값:  껙 출력 값:  껙
, 실제 값:  과 출력 값:  과
, 실제 값:  웽 출력 값:  웽
, 실제 값:  깝 출력 값:  깝
, 실제 값:  쩠 출력 값:  쩠
, 실제 값:  짖 출력 값:  짖
, 실제 값:  슉 출력 값:  슉
, 실제 값:  쵯 출력 값:  쵯
, 실제 값:  뤽 출력 값:  뤽
, 실제 값:  쩨 출력 값:  쩨
, 실제 값:  뛸 출력 값:  뛸
, 실제 값:  랍 출력 값:  랍
, 실제 값:  더 출력 값:  더
, 실제 값:  삼 출력 값:  삼
, 실제 값:  랐 출력 값:  랐
, 실제 값:  븅 출력 값:  븅
, 실제 값:  폐 출력 값:  폐
, 실제 값:  빵 출력 값:  빵
, 실제 값:  셕 출력 값:  셕
, 실제 값:  켑 출력 값:  켑
, 실제 값:  귐 출력 값:  귐
, 실제 값:  샹 출력 값:  샹
, 실제 값:  죵 출력 값:  죵
, 실제 값:  튿 출력 값:  튿


, 실제 값:  뗏 출력 값:  뗏
, 실제 값:  툉 출력 값:  툉
, 실제 값:  울 출력 값:  울
, 실제 값:  옴 출력 값:  옴
, 실제 값:  핌 출력 값:  핌
, 실제 값:  있 출력 값:  있
, 실제 값:  갓 출력 값:  갓
, 실제 값:  쏨 출력 값:  쏨
, 실제 값:  틉 출력 값:  틉
, 실제 값:  리 출력 값:  리
, 실제 값:  츔 출력 값:  츔
, 실제 값:  은 출력 값:  은
, 실제 값:  헌 출력 값:  헌
, 실제 값:  듐 출력 값:  듐
, 실제 값:  댄 출력 값:  댄
, 실제 값:  렇 출력 값:  렇
, 실제 값:  틘 출력 값:  틘
, 실제 값:  믈 출력 값:  믈
, 실제 값:  웠 출력 값:  웠
, 실제 값:  갹 출력 값:  갹
, 실제 값:  굻 출력 값:  굻
, 실제 값:  쏙 출력 값:  쏙
, 실제 값:  숄 출력 값:  숄
, 실제 값:  검 출력 값:  검
, 실제 값:  렙 출력 값:  렙
, 실제 값:  헨 출력 값:  헨
, 실제 값:  꺾 출력 값:  꺾
, 실제 값:  신 출력 값:  신
, 실제 값:  갠 출력 값:  갠
, 실제 값:  밧 출력 값:  밧
, 실제 값:  쫘 출력 값:  쫘
, 실제 값:  칩 출력 값:  칩
, 실제 값:  쒀 출력 값:  쒀
, 실제 값:  박 출력 값:  박
, 실제 값:  킹 출력 값:  킹
, 실제 값:  쫌 출력 값:  쫌
, 실제 값:  휠 출력 값:  휠
, 실제 값:  팟 출력 값:  팟
, 실제 값:  깻 출력 값:  깻
, 실제 값:  밭 출력 값:  밭
, 실제 값:  홋 출력 값:  홋
, 실제 값:  깽 출력 값:  깽
, 실제 값:  찡 출력 값:  찡
, 실제 값:  줴 출력 값:  줴
, 실제 값:  꾸 출력 값:  꾸
, 실제 값:  옘 출력 값:  옘
, 실제 값:  쏭 출력 값:  쏭
, 실제 값:  쐼 출력 값:  쐼
, 실제 값:  색 출력 값:  색
, 실제 값:  뺌 출력 값:  뺌


...

, 실제 값:  쌌 출력 값:  쌌
, 실제 값:  접 출력 값:  접
, 실제 값:  팠 출력 값:  팠
, 실제 값:  쐤 출력 값:  쐤
, 실제 값:  웹 출력 값:  웹
, 실제 값:  흰 출력 값:  흰
, 실제 값:  곯 출력 값:  곯
, 실제 값:  뇟 출력 값:  뇟
, 실제 값:  휭 출력 값:  휭
, 실제 값:  갓 출력 값:  갓
, 실제 값:  떤 출력 값:  떤
, 실제 값:  킷 출력 값:  킷
, 실제 값:  뱄 출력 값:  뱄
, 실제 값:  쥐 출력 값:  쥐
, 실제 값:  못 출력 값:  못
, 실제 값:  펫 출력 값:  펫
, 실제 값:  틀 출력 값:  틀
, 실제 값:  곶 출력 값:  곶
, 실제 값:  몄 출력 값:  몄
, 실제 값:  삿 출력 값:  삿
, 실제 값:  렁 출력 값:  렁
, 실제 값:  걔 출력 값:  걔
, 실제 값:  뻘 출력 값:  뻘
, 실제 값:  타 출력 값:  타
, 실제 값:  넘 출력 값:  넘
, 실제 값:  쭐 출력 값:  쭐
, 실제 값:  뇨 출력 값:  뇨
, 실제 값:  쒼 출력 값:  쒼
, 실제 값:  으 출력 값:  으
, 실제 값:  숩 출력 값:  숩
, 실제 값:  뭡 출력 값:  뭡
, 실제 값:  닺 출력 값:  닺
, 실제 값:  헴 출력 값:  헴
, 실제 값:  뽐 출력 값:  뽐
, 실제 값:  삡 출력 값:  삡
, 실제 값:  진 출력 값:  진
, 실제 값:  랏 출력 값:  랏
, 실제 값:  팻 출력 값:  팻
, 실제 값:  싫 출력 값:  싫
, 실제 값:  굔 출력 값:  굔
, 실제 값:  렁 출력 값:  렁
, 실제 값:  꽹 출력 값:  꽹
, 실제 값:  좝 출력 값:  좝
, 실제 값:  랖 출력 값:  랖
, 실제 값:  슁 출력 값:  슁
, 실제 값:  턺 출력 값:  턺
, 실제 값:  쿰 출력 값:  쿰
, 실제 값:  딤 출력 값:  딤
, 실제 값:  쭤 출력 값:  쭤
, 실제 값:  갱 출력 값:  갱


, 실제 값:  욀 출력 값:  욀
, 실제 값:  팜 출력 값:  팜
, 실제 값:  맥 출력 값:  맥
, 실제 값:  굄 출력 값:  굄
, 실제 값:  꿴 출력 값:  꿴
, 실제 값:  튱 출력 값:  튱
, 실제 값:  셸 출력 값:  셸
, 실제 값:  봉 출력 값:  봉
, 실제 값:  뤼 출력 값:  뤼
, 실제 값:  꿈 출력 값:  꿈
, 실제 값:  쾀 출력 값:  쾀
, 실제 값:  슈 출력 값:  슈
, 실제 값:  흗 출력 값:  흗
, 실제 값:  맣 출력 값:  맣
, 실제 값:  듕 출력 값:  듕
, 실제 값:  췻 출력 값:  췻
, 실제 값:  의 출력 값:  의
, 실제 값:  쉽 출력 값:  쉽
, 실제 값:  난 출력 값:  난
, 실제 값:  쒀 출력 값:  쒀
, 실제 값:  곰 출력 값:  곰
, 실제 값:  뿌 출력 값:  뿌
, 실제 값:  슐 출력 값:  슐
, 실제 값:  린 출력 값:  린
, 실제 값:  옳 출력 값:  옳
, 실제 값:  옻 출력 값:  옻
, 실제 값:  텀 출력 값:  텀
, 실제 값:  궝 출력 값:  궝
, 실제 값:  뇌 출력 값:  뇌
, 실제 값:  험 출력 값:  험
, 실제 값:  굴 출력 값:  굴
, 실제 값:  뛴 출력 값:  뛴
, 실제 값:  솨 출력 값:  솨
, 실제 값:  팃 출력 값:  팃
, 실제 값:  덩 출력 값:  덩
, 실제 값:  떻 출력 값:  떻
, 실제 값:  괆 출력 값:  괆
, 실제 값:  큰 출력 값:  큰
, 실제 값:  수 출력 값:  수
, 실제 값:  산 출력 값:  산
, 실제 값:  뇩 출력 값:  뇩
, 실제 값:  싼 출력 값:  싼
, 실제 값:  빎 출력 값:  빎
, 실제 값:  챔 출력 값:  챔
, 실제 값:  왠 출력 값:  왠
, 실제 값:  얄 출력 값:  얄
, 실제 값:  뎃 출력 값:  뎃
, 실제 값:  잠 출력 값:  잠
, 실제 값:  엣 출력 값:  엣
, 실제 값:  읠 출력 값:  읠


, 실제 값:  것 출력 값:  것
, 실제 값:  멥 출력 값:  멥
, 실제 값:  쩨 출력 값:  쩨
, 실제 값:  푭 출력 값:  푭
, 실제 값:  났 출력 값:  났
, 실제 값:  듣 출력 값:  듣
, 실제 값:  듐 출력 값:  듐
, 실제 값:  항 출력 값:  항
, 실제 값:  쏵 출력 값:  쏵
, 실제 값:  쿳 출력 값:  쿳
, 실제 값:  땐 출력 값:  땐
, 실제 값:  철 출력 값:  철
, 실제 값:  붐 출력 값:  붐
, 실제 값:  괘 출력 값:  괘
, 실제 값:  쵭 출력 값:  쵭
, 실제 값:  저 출력 값:  저
, 실제 값:  챗 출력 값:  챗
, 실제 값:  헴 출력 값:  헴
, 실제 값:  복 출력 값:  복
, 실제 값:  썹 출력 값:  썹
, 실제 값:  린 출력 값:  린
, 실제 값:  닺 출력 값:  닺
, 실제 값:  왕 출력 값:  왕
, 실제 값:  랐 출력 값:  랐
, 실제 값:  븟 출력 값:  븟
, 실제 값:  쑹 출력 값:  쑹
, 실제 값:  늠 출력 값:  늠
, 실제 값:  쇼 출력 값:  쇼
, 실제 값:  습 출력 값:  습
, 실제 값:  큭 출력 값:  큭
, 실제 값:  겔 출력 값:  겔
, 실제 값:  았 출력 값:  았
, 실제 값:  앗 출력 값:  앗
, 실제 값:  벡 출력 값:  벡
, 실제 값:  쌕 출력 값:  쌕
, 실제 값:  콜 출력 값:  콜
, 실제 값:  앍 출력 값:  앍
, 실제 값:  읠 출력 값:  읠
, 실제 값:  뜬 출력 값:  뜬
, 실제 값:  향 출력 값:  향
, 실제 값:  먈 출력 값:  먈
, 실제 값:  핍 출력 값:  핍
, 실제 값:  튼 출력 값:  튼
, 실제 값:  흣 출력 값:  흣
, 실제 값:  톼 출력 값:  톼
, 실제 값:  챙 출력 값:  챙
, 실제 값:  딨 출력 값:  딨
, 실제 값:  뼁 출력 값:  뼁
, 실제 값:  암 출력 값:  암
, 실제 값:  단 출력 값:  단


, 실제 값:  뒈 출력 값:  뒈
, 실제 값:  룩 출력 값:  룩
, 실제 값:  짜 출력 값:  짜
, 실제 값:  율 출력 값:  율
, 실제 값:  권 출력 값:  권
, 실제 값:  템 출력 값:  템
, 실제 값:  옇 출력 값:  옇
, 실제 값:  꿀 출력 값:  꿀
, 실제 값:  범 출력 값:  범
, 실제 값:  굶 출력 값:  굶
, 실제 값:  훌 출력 값:  훌
, 실제 값:  샵 출력 값:  샵
, 실제 값:  쾌 출력 값:  쾌
, 실제 값:  륩 출력 값:  륩
, 실제 값:  촹 출력 값:  촹
, 실제 값:  멨 출력 값:  멨
, 실제 값:  탭 출력 값:  탭
, 실제 값:  껭 출력 값:  껭
, 실제 값:  닷 출력 값:  닷
, 실제 값:  꼼 출력 값:  꼼
, 실제 값:  얩 출력 값:  얩
, 실제 값:  쓴 출력 값:  쓴
, 실제 값:  늄 출력 값:  늄
, 실제 값:  엊 출력 값:  엊
, 실제 값:  딧 출력 값:  딧
, 실제 값:  끌 출력 값:  끌
, 실제 값:  렷 출력 값:  렷
, 실제 값:  단 출력 값:  단
, 실제 값:  뻑 출력 값:  뻑
, 실제 값:  좝 출력 값:  좝
, 실제 값:  휘 출력 값:  휘
, 실제 값:  졌 출력 값:  졌
, 실제 값:  쟉 출력 값:  쟉
, 실제 값:  묍 출력 값:  묍
, 실제 값:  뭄 출력 값:  뭄
, 실제 값:  괍 출력 값:  괍
, 실제 값:  순 출력 값:  순
, 실제 값:  뭘 출력 값:  뭘
, 실제 값:  겯 출력 값:  겯
, 실제 값:  뫙 출력 값:  뫙
, 실제 값:  쨈 출력 값:  쨈
, 실제 값:  읏 출력 값:  읏
, 실제 값:  떱 출력 값:  떱
, 실제 값:  굻 출력 값:  굻
, 실제 값:  돐 출력 값:  돐
, 실제 값:  껴 출력 값:  껴
, 실제 값:  둑 출력 값:  둑
, 실제 값:  걸 출력 값:  걸
, 실제 값:  홴 출력 값:  홴
, 실제 값:  핥 출력 값:  핥


, 실제 값:  튼 출력 값:  튼
, 실제 값:  뭇 출력 값:  뭇
, 실제 값:  죽 출력 값:  죽
, 실제 값:  탱 출력 값:  탱
, 실제 값:  뼙 출력 값:  뼙
, 실제 값:  쉔 출력 값:  쉔
, 실제 값:  스 출력 값:  스
, 실제 값:  컸 출력 값:  컸
, 실제 값:  촘 출력 값:  촘
, 실제 값:  얇 출력 값:  얇
, 실제 값:  딪 출력 값:  딪
, 실제 값:  꾸 출력 값:  꾸
, 실제 값:  뫈 출력 값:  뫈
, 실제 값:  슭 출력 값:  슭
, 실제 값:  훼 출력 값:  훼
, 실제 값:  뭇 출력 값:  뭇
, 실제 값:  뱃 출력 값:  뱃
, 실제 값:  쇠 출력 값:  쇠
, 실제 값:  곶 출력 값:  곶
, 실제 값:  츳 출력 값:  츳
, 실제 값:  옇 출력 값:  옇
, 실제 값:  륄 출력 값:  륄
, 실제 값:  큐 출력 값:  큐
, 실제 값:  쨌 출력 값:  쨌
, 실제 값:  횹 출력 값:  횹
, 실제 값:  가 출력 값:  가
, 실제 값:  몹 출력 값:  몹
, 실제 값:  뻥 출력 값:  뻥
, 실제 값:  슉 출력 값:  슉
, 실제 값:  데 출력 값:  데
, 실제 값:  츙 출력 값:  츙
, 실제 값:  꼍 출력 값:  꼍
, 실제 값:  졍 출력 값:  졍
, 실제 값:  춈 출력 값:  춈
, 실제 값:  흼 출력 값:  흼
, 실제 값:  규 출력 값:  규
, 실제 값:  초 출력 값:  초
, 실제 값:  려 출력 값:  려
, 실제 값:  긱 출력 값:  긱
, 실제 값:  뎄 출력 값:  뎄
, 실제 값:  윗 출력 값:  윗
, 실제 값:  냠 출력 값:  냠
, 실제 값:  빌 출력 값:  빌
, 실제 값:  먼 출력 값:  먼
, 실제 값:  읕 출력 값:  읕
, 실제 값:  팍 출력 값:  팍
, 실제 값:  온 출력 값:  온
, 실제 값:  짊 출력 값:  짊
, 실제 값:  섶 출력 값:  섶
, 실제 값:  끝 출력 값:  끝


In [52]:
sum=0
for i in range(len(scoreboard)):
    sum+=int(scoreboard[i])
performance=sum/len(scoreboard)*100
print("performance: ",performance,"%")

performance:  99.998226918917 %
